Import Libraries/Data

In [1]:
import os
import sys
import glob
import numpy as np
import torch
from torchvision import transforms, datasets
from PIL import Image

#Import image loaders
os.chdir("../../src/")
from load_data import preview_images, load_images

In [2]:
#Import and preview image data
train_path= "../Data/train/"
horse_train= load_images(train_path + "horses/")
human_train= load_images(train_path + "humans/")

val_path= "../Data/validation/"
horse_test= load_images(val_path + "horses/")
human_test= load_images(val_path + "humans/")

#Sanity Check
print("Training Horse Data: %s \nTest Horse Data: %s\n" % (horse_train.shape, horse_test.shape))
print("Training Human Data: %s \nTest Human Data: %s" % (human_train.shape, human_test.shape))

Training Horse Data: (500, 300, 300, 4) 
Test Horse Data: (128, 300, 300, 4)

Training Human Data: (527, 300, 300, 4) 
Test Human Data: (128, 300, 300, 4)


Preprocess **Training** Data

In [3]:
#Assign labels: 1= human, 0 = horse
y_train_human= np.full((len(human_train),1), 1)
y_train_horse= np.full((len(horse_train),1), 0)

#Concatenate Training data
X= np.concatenate((horse_train, human_train), axis=0)
y= np.concatenate((y_train_horse, y_train_human), axis= 0)

#Zero-center data
training_mean= X.mean()
training_std= X.std()
X= (X - training_mean)/training_std

#Shuffle and convert to tensors
from sklearn.utils import shuffle
X_train, y_train= shuffle(X, y)

X_train= torch.from_numpy(X_train)
y_train= torch.from_numpy(y_train)

#Sanity check
print("Training data shape: %s\nData type: %s \nTraining label shape: %s\nData type: %s" % (X_train.shape,
                                                                                              type(X_train),
                                                                                              y_train.shape,
                                                                                              type(y_train)))

Training data shape: torch.Size([1027, 300, 300, 4])
Data type: <class 'torch.Tensor'> 
Training label shape: torch.Size([1027, 1])
Data type: <class 'torch.Tensor'>


# Modeling

Base ANN 

In [4]:
#Use base model from early computer vision
from models import BaseNeuralNetwork

device= "cpu"
base_model= BaseNeuralNetwork().to(device)
print(base_model)

BaseNeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=360000, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=2, bias=True)
  )
)


Base ANN training

In [11]:
from torch import optim
from torch import nn
from torch.utils.data import DataLoader


#Set up loss function, optimizer and batch
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(base_model.parameters(),
                      lr= 10 ** -3,
                       momentum= .9)

batch_size= 32
#data loader allows for batching and provides iterable over dataset
train_loader= DataLoader(X_train,
                                         batch_size,
                                         shuffle= True)


                                        

In [14]:
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)

In [12]:
# get some random training images
dataiter = iter(train_loader)
images, labels = dataiter.next()

ValueError: too many values to unpack (expected 2)

In [ ]:
train_features, train_labels = next(iter(train_dataset))


In [25]:
train_labels[0:10]

tensor([0])

In [ ]:
adfadfafaf

In [ ]:
data_batch, labels_batch = next(iter(train_loader)), next(iter(train_loader))
print(data_batch.size())


print(labels_batch.size())

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size= len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        X, y= X.to(device), y.to(device)
        #Compute prediction error
        pred= model(X)
        loss= loss_fn(pred, y)
        #Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 100 == 0:
            loss, current= loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
epochs= 5

for e in range(epochs):
    epoch_loss= 0.0
    
for batch_size, (X_train, y_train) in enumerate(train_loader):
    
    optimizer.zero_grad() #sets gradient to 0
    
    preds = base_model(X_train) #perform forward pass
    loss = criterion(preds, y_train) #calculate loss from preds from ground truth
    loss.backward()
#    print("Training Epoch:%s\n--------------------------------------" % (e + 1))#

In [ ]:
dont run below this line 

In [ ]:
logits= base_model(X.float())

In [ ]:
logits.shape

In [ ]:
pred_probs= torch.nn.Softmax(dim= 1)(logits)

In [ ]:
y_pred = pred_probs.argmax(1)
print(f"Predicted class: {y_pred}")


In [ ]:
#calculating accuracy between predictions and truth
y_train

In [ ]:
#PyTorch CNN

**Deploy**

In [ ]:
# y_test_human= np.full((len(human_test),1), 1)
# y_test_horse= np.full((len(horse_test),1), 0)

# #Concatenate validation set
# X= np.concatenate((horse_test, human_test), axis=0)
# y= np.concatenate((y_test_horse, y_test_human), axis= 0)

# #Zero-Center data
# X= (X- training_mean)/training_std

# #Shuffle
# X_test, y_test= shuffle(X, y)

# #Sanity check
# print("Test data shape: %s \n Training label shape: %s" % (X_test.shape, y_test.shape))